# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902065


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:04<01:02,  2.16s/it]

Rendering models:  10%|▉         | 3/31 [00:04<00:43,  1.55s/it]

Rendering models:  16%|█▌        | 5/31 [00:04<00:28,  1.11s/it]

Rendering models:  19%|█▉        | 6/31 [00:07<00:40,  1.64s/it]

...

Rendering models:  87%|████████▋ | 27/31 [00:10<00:00,  6.45it/s]

Rendering models:  90%|█████████ | 28/31 [00:10<00:00,  6.69it/s]

Rendering models:  94%|█████████▎| 29/31 [00:10<00:00,  5.62it/s]

Rendering models: 100%|██████████| 31/31 [00:11<00:00,  7.12it/s]

equidad1                              0.000141
not-logged-in-cae133427d99aac1e554    0.000109
kayleebug2017                         0.000478
JessieDawn                            0.000142
ElisabethB                            0.000144
vosskftw                              0.006128
not-logged-in-f5061ed7c6ff388b798c    0.005012
bbergerud                             0.000140
Thedunkmasta                          0.000241
Chelskee                              0.000090
not-logged-in-7526601586eaa938b3ee    0.000279
Synapsestar                           0.000135
not-logged-in-1df2dc33157510bbec3e    0.000158
crush202020                           0.000120
quirito19                             0.000118
aidenr                                0.000345
not-logged-in-f87600b62b6963cef865    0.001169
not-logged-in-68c93388f62f1185dba6    0.000135
not-logged-in-986357d841d09bf87b26    0.243127
LeusaneLordelo                        0.000207
TortleBoi                             0.045090
rmacmi2      

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())